In [1]:
# Importing packages
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
from scipy.optimize import basinhopping

# Importing Pauli matrices
from pauli_matrices.pauli_matrices import I, X, Y, Z

# Importing figure properties
from figures_report.figure_properties import *

# Importing functions
from Bell_operator.Bell_operator import calc_Bell_operator
from classical_optimization.classical_bound import calc_classical_bound

# Defining saving directory
save_dir = '../data/figures/toy_model/'      # figures
save_fdir = '../data/data_sets/toy_model/'   # Files

In [2]:
# Defining the number of qubits
N = 2

# Defining the Hamiltonian
H = np.sqrt(2)*(np.kron(Z, Z) + np.kron(X, X)) 

# Calculating the eigenvalues and eigenstates
eig_vals, eig_vecs = np.linalg.eigh(H)

# extracting the ground state energy and the respective eigenstate
psi_G = np.zeros((2**N, 1))
psi_G[:,0] = eig_vecs[:,0]
rho = np.matmul(psi_G, np.transpose(psi_G))

In [13]:
# Defining number of measurements and the angles of the N*m-1 measurements
m = nb.typed.List(
    [2, 2]
)

# Defining measurements
theta = [np.pi/4]

# Initializing basis and extra Z gate
basis = 'XZ'
extra_gate = False

# Calculating the classical bound
beta_C = calc_classical_bound(theta, H, N, m, basis, extra_gate)
print('Classical bound: %.16f' %(beta_C))

# Calculating eigenvalue of the Bell operator
beta_Q= calc_Bell_operator(theta, H, rho, N, m, basis, extra_gate)
print('Quantum bound: %.16f' %(beta_Q))


Classical bound: -2.0000000000000004
Quantum bound: -2.8284271247461894


In [20]:
# Defining the number of measurements
m = nb.typed.List(
    [2, 2]
)

# Initializing empty list beta_C for call_back function
beta_C = []
stored_angles = []

# Defining angles
eps = 1e-2
init_guess = np.random.rand(1)*(np.pi-2*eps)+eps

# Defining arguments
basis = 'XZ'
extra_Z_gate = False
args = (H, N, m, basis, extra_Z_gate)

# Defining minimization options
options = {
        'maxiter'  : 2000,
}

# Defining a fitting function and a callback function
def fitting_func(theta, H, N, m, basis, extra_Z_gate):

    return -calc_classical_bound(theta, H, N, m, basis, extra_Z_gate)


def callback_function(theta):

    stored_angles.append( theta )
    beta_C.append( calc_classical_bound(theta, H, N, m, basis, extra_Z_gate) )


# Defining the minimizer keyword arguments
minimizer_kwargs = {
                'method'  : "Nelder-Mead",
                'args'    : args,
                'options' : options,
                'callback': callback_function
}

res = basinhopping(fitting_func, init_guess,
                minimizer_kwargs = minimizer_kwargs,
                niter = 10,
                # disp = True
)

np.save(save_fdir+'beta_C.npy', beta_C)
np.save(save_fdir+'theta.npy', stored_angles)
res

                        fun: 2.0000022231182717
 lowest_optimization_result:  final_simplex: (array([[0.78539705],
       [0.78543326]]), array([2.00000222, 2.00007018]))
           fun: 2.0000022231182717
       message: 'Optimization terminated successfully.'
          nfev: 22
           nit: 11
        status: 0
       success: True
             x: array([0.78539705])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 288
                        nit: 10
                          x: array([0.78539705])